In [96]:
import numpy as np
import random
from datetime import datetime
import time
from collections import defaultdict
from helper_func import *
import pickle
import multiprocessing
import keras
from multiprocessing import Pool
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from collections import deque
from keras.activations import relu, linear
from keras.losses import mean_squared_error
from keras.optimizers import Adam
import tensorflow as tf

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [97]:
# The inputs are 28x28x28 volumes with a single channel, and the  
# batch size is 4  
input_shape =(4, 28, 28, 28, 1)
x = tf.random.normal(input_shape)
y = tf.keras.layers.Conv3D(2, 3, activation='relu', input_shape=input_shape[1:])(x)
print(y)
print(y.shape)


Tensor("conv3d_3/Relu:0", shape=(4, 26, 26, 26, 2), dtype=float32)
(4, 26, 26, 26, 2)


In [98]:
%autoreload 2

In [99]:
class pentago:
    """
    
    """

    def __init__(self, state = None):
        """Initializes the class reservation"""
        #print('initializing')
        
        if state == None:
            self.state = state = np.zeros((6,6), dtype=np.int)
        self.history = []
        self.winner = None
        self.gameover = False
        self.player_turn = 1
    
    def current_board_state(self):
        # need to return a copy or bad stuff happens
        return copy.copy(self.state)
    
    def game_history(self, player, move, cuad, rotatation):
        self.history.append((boardstate_to_ideal_key(self.state), ideal_state(self.state), player, move, cuad, rotatation))
        #return self.history

    def find_winner(self, board_state):
        player1_win = False
        player_min1_win = False
        diagonal1 = board_state.diagonal()
        diagonal2 = np.fliplr(board_state).diagonal()
        winning_slices =  np.vstack([board_state[1:,:].T, board_state[:-1,:].T, # all columns
                              board_state[:,1:], board_state[:,:-1], # all rows
                              diagonal1[1:], diagonal1[:-1], # diagonal 1
                              diagonal2[1:],diagonal2[1:], # diagonal 2
                              board_state.diagonal(offset=1), board_state.diagonal(offset=-1), # diagonal offsets 
                              np.fliplr(board_state).diagonal(offset=1), np.fliplr(board_state).diagonal(offset=-1)] ) # diagonal offsets
        sums = np.dot(winning_slices, np.array([1,1,1,1,1]))
        if 5 in sums: player1_win = True
        if -5 in sums: player_min1_win = True
        if player1_win == True or player_min1_win == True:
           # print("Player 1 winner?", player1_win, "Player -1 winner?", player_min1_win)
            self.gameover = True
            if player1_win == True:
                self.winner = 1
            elif player_min1_win ==True:
                self.winner = -1
            self.history.append(self.winner)
        return "Win"

    def check_gameover(self):
        if not 0 in self.state:
              self.gameover = True
              #print("The game board is full!")
        
    def full_move(self, move, cuad, direction, player, dtype=np.int):
        if player != self.player_turn:
            print( "error, wrong player turn. No move taken.")
            return 'Error, wrong player turn.'
        self.state = fullmove(self.state,move, cuad, direction, player)


        self.game_history(move, player, cuad, direction)
        self.find_winner(self.state) #return in find_winner if a winner is found
        self.check_gameover() #return in check_gameover
        if player == 1:
            self.player_turn = -1
        else:
            self.player_turn = 1
        #print('Successful Move')
        return self.state



In [100]:
class q_table:

    def __init__(self,length=0, games_played=0):
        """Initializes the class reservation"""
        self.time = datetime.now()
        self.length = length
        self.q_dict = {}
        self.games_played = games_played

  #def time(self):
    #self.time = time

    def length(self):
        self.length += 1
    #self.length = length  
    
    def get_q_value(self, boardstate):
        return self.q_dict.get(boardstate, (0, 0))
    
    def update_q_value(self, boardstate, new_val, update_function = None):
        q_val, n = self.get_q_value(boardstate) 
        if update_function:
            #print('using custom function')
            self.q_dict[boardstate] = update_function(q_val, n, new_val)
        else:
            self.q_dict[boardstate] = [new_val, n+1]
        return self.q_dict[boardstate]
    
    def update_post_game(self, history, update_fn):
        winner = history[-1]
        
        for boardposition in history[-2::-1]:
            key = boardposition[0]
            #print(key, winner)
            self.update_q_value(key, winner, update_fn)

    def update_post_game2(self, history, update_fn, decay_reward = .9):
        winner = history[-1]
        
        for boardposition in history[-2::-1]:
            key = boardposition[0]
            #print(key, winner)
            self.update_q_value(key, winner, update_fn)
            winner *= decay_reward

    
    

In [130]:
class cnn_model:

    def __init__(self,list_density=None,lr=0.02, weights = None):
        self.model = self.build_model(list_density,lr)
        if weights:
            self.model.load_weights(weights)

    def build_model(self,den,lr):
        model = Sequential()
        #model.add(tf.keras.Input(shape=(6,6,2,)))
        #for layer in den:
        model.add(Conv2D(
                            filters = 64, 
                            kernel_size = (3,3),
                            strides = (3,3),
                            padding = 'valid',
                            activation = relu,
                            input_shape = (6,6,2)))
        model.add(Flatten())
        model.add(Dense(256, activation = relu))
        model.add(Dense(128, activation = relu))


        model.add(Dense(1, activation=linear))
        
        model.compile(loss=mean_squared_error,optimizer=Adam(lr=lr))
        print(model.summary())
        return model



    def update_model(self,states_batch, q_batch, epochs = 1):

        self.model.fit(states_batch,q_batch,epochs = epochs, verbose=1)

    def predict_model(self, state_batch):
        return self.model.predict_on_batch(state_batch)
        

In [102]:
c = cnn_model()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 2, 2, 64)          1216      
_________________________________________________________________
flatten_14 (Flatten)         (None, 256)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_41 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 129       
Total params: 100,033
Trainable params: 100,033
Non-trainable params: 0
_________________________________________________________________
None


In [103]:
x = np.array([[1,1,1,-1,1,1],[0,0,0,0,0,0],[0,0,0,1,1,0],[0,0,1,-1,-1,0],[0,0,1,-1,1,0],[-1,-1,1,0,0,0]])
x = boardstate_to_cnn_input(x)
print(x.shape)
out= c.model.predict(x.reshape(1,6,6,2))
print(out.shape)
out

(6, 6, 2)
(1, 1)


array([[0.06724047]], dtype=float32)

In [111]:
class nn_agent:
    
    def __init__(self, nn, player = 1, epsilon = 1, epsilon_decay = .99995, epsilon_min = .5):
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.nn = nn
        self.player = player
        self.epsilon_min = epsilon_min
        
            
    def get_avail_moves(self,boardstate):
        """
        This method creates a list with available spaces in the board and combination of quadrant and rotation
        The input is the board state (6x6) numpy array
        """
        x = np.where(boardstate == 0)
        #print(x)
        available_positions_for_placement = list(zip(x[0], x[1]))
        
        # all available positions (p), quadrants(q), rotations(r)
        available_moves = [(p,q,r) for p in available_positions_for_placement for q in [1,2,3,4] for r in [-1,1]]
        #print(len(available_moves))
        return available_moves
    
    def get_possible_next_boardstates(self, boardstate):
        next_possible_boardstates = defaultdict(list)
        for move in self.get_avail_moves(boardstate):
            possible_boardstate = fullmove(boardstate,*move, self.player)
            #print(possible_boardstate)
            key = boardstate_to_ideal_key(possible_boardstate)
            cnn_input = boardstate_to_cnn_input(possible_boardstate)
            #print(cnn_input)
            #print(key)
            next_possible_boardstates[key].append((cnn_input, move))
            
        return next_possible_boardstates
    
    def make_move(self, game):
        
        # get the current boardstate from the pentago class
        boardstate = game.current_board_state()
        
        # get possible next possible boardstates
        #t0 = time.time()
        next_possible_boardstates = self.get_possible_next_boardstates(boardstate)
        key_list = list(next_possible_boardstates.keys())
        #print("get possible boardstates", time.time()-t0)
        # determine if to take random move
        if np.random.rand() < self.epsilon:
            random_bs = random.choice(key_list)
            random_mv = next_possible_boardstates[random_bs][0][1] # capture the move needed
            
            game.full_move(*random_mv,self.player)
            
        else:
            #print("not random", self.player)
            #t0 = time.time()
            nn_input_boardstate = []
            move = []
            for k,v in next_possible_boardstates.items():
                nn_input_boardstate.append(v[0][0])
                #print(v[0][0].shape)
                move.append(v[0][1])
          
            #print(nn_input_boardstate, type(nn_input_boardstate))
            #nn_input_batch = np.concatenate(nn_input_boardstate)
            nn_input_batch = np.array(nn_input_boardstate)
            #print(nn_input_batch, type(nn_input_batch))
            #print(nn_input_boardstate)
            q_values = self.nn.predict_model(nn_input_batch)
            #print("get q_values boardstates", time.time()-t0)

            #print(q_values, type(q_values))
            #print("----")
            q_values *= self.player
            #q_values_list = [x*self.player for x in list(q_values)]
            #print(q_values_list)
            # get random index of a max value
            #print('length fo q_values_list = ', len(q_values_list))
            max_q = np.argmax(q_values)
            #index_of_all_max = [i for i in range(len(q_values)) if q_values[i] == max_q]
            #random_max_q_index = random.choice(index_of_all_max)
            #print("MAX VALUE:", q_values_list[random_max_q_index])
            mv_to_take = move[max_q]
            game.full_move(*mv_to_take, self.player)
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay 
        else:
            self.epsilon = self.epsilon_min

In [112]:
class nn_agent_multigame:
    
    def __init__(self, nn, player = 1, epsilon = 1, epsilon_decay = .99995, epsilon_min = .5):
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.nn = nn
        self.player = player
        self.epsilon_min = epsilon_min
        
            
    def get_avail_moves(self,boardstate):
        """
        This method creates a list with available spaces in the board and combination of quadrant and rotation
        The input is the board state (6x6) numpy array
        """
        x = np.where(boardstate == 0)
        #print(x)
        available_positions_for_placement = list(zip(x[0], x[1]))
        
        # all available positions (p), quadrants(q), rotations(r)
        available_moves = [(p,q,r) for p in available_positions_for_placement for q in [1,2,3,4] for r in [-1,1]]
        #print(len(available_moves))
        return available_moves
    
    def get_possible_next_boardstates(self, boardstate):
        next_possible_boardstates = defaultdict(list)
        for move in self.get_avail_moves(boardstate):
            possible_boardstate = fullmove(boardstate,*move, self.player)
            #print(possible_boardstate)
            key = boardstate_to_ideal_key(possible_boardstate)
            cnn_input = boardstate_to_cnn_input(possible_boardstate)
            #print(cnn_input)
            #print(key)
            next_possible_boardstates[key].append((cnn_input, move))
            
        return next_possible_boardstates
    
    def make_move(self, game):
        
        # get the current boardstate from the pentago class
        boardstate = game.current_board_state()
        
        # get possible next possible boardstates
        next_possible_boardstates = self.get_possible_next_boardstates(boardstate)
        key_list = list(next_possible_boardstates.keys())
        
        # determine if to take random move
        if np.random.rand() < self.epsilon:
            random_bs = random.choice(key_list)
            random_mv = next_possible_boardstates[random_bs][0][1] # capture the move needed
            
            game.full_move(*random_mv,self.player)
            
        else:
            #print("not random", self.player)
            nn_input_boardstate = []
            move = []
            for k,v in next_possible_boardstates.items():
                nn_input_boardstate.append(v[0][0])
                #print(v[0][0].shape)
                move.append(v[0][1])
            
            
            #print(nn_input_boardstate, type(nn_input_boardstate))
            #nn_input_batch = np.concatenate(nn_input_boardstate)
            nn_input_batch = np.array(nn_input_boardstate)
            #print(nn_input_batch, type(nn_input_batch))
            #print(nn_input_boardstate)
            q_values = self.nn.predict_model(nn_input_batch)
            #print(q_values, type(q_values))
            #print("----")
            q_values *= self.player
            #q_values_list = [x*self.player for x in list(q_values)]
            #print(q_values_list)
            # get random index of a max value
            #print('length fo q_values_list = ', len(q_values_list))
            max_q = np.argmax(q_values)
            #index_of_all_max = [i for i in range(len(q_values)) if q_values[i] == max_q]
            #random_max_q_index = random.choice(index_of_all_max)
            #print("MAX VALUE:", q_values_list[random_max_q_index])
            mv_to_take = move[max_q]
            game.full_move(*mv_to_take, self.player)
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay 
        else:
            self.epsilon = self.epsilon_min

In [113]:
x = np.array([1,2,3,4,2,1,4])
np.argmax(x)

3

In [117]:
with open('CNN_1_2020-12-05 18_36_18.pickle', 'rb') as file:
    nn_1 = pickle.load(file) 

In [129]:
nn_1.model.save_weights('weights.h5')

In [145]:
nn_2 = cnn_model(weights='weights.h5')

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 2, 2, 64)          1216      
_________________________________________________________________
flatten_19 (Flatten)         (None, 256)               0         
_________________________________________________________________
dense_55 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_56 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_57 (Dense)             (None, 1)                 129       
Total params: 100,033
Trainable params: 100,033
Non-trainable params: 0
_________________________________________________________________
None


In [146]:
agent1 = nn_agent(player = 1,  nn = nn_1, epsilon_min = 0, epsilon_decay = .999, epsilon = 0)
agent2 = nn_agent(player = -1,  nn = nn_2, epsilon_min = 0, epsilon = 0)

In [147]:
t0 = time.time()
g = pentago()
while g.gameover == False:
    agent1.make_move(g)
    if g.gameover == True: break
    agent2.make_move(g)
    #break

#bs = agent1.get_possible_next_boardstates(g.current_board_state())
    
print(time.time()-t0)
print(len(g.history))
g.history

2.2280657291412354
28


[('[000000000000000000000000000000000100]',
  array([[0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0]]),
  (0, 0),
  1,
  1,
  -1),
 ('[000000000000000000000100000000000200]',
  array([[ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  1,  0,  0],
         [ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0, -1,  0,  0]]),
  (0, 0),
  -1,
  1,
  -1),
 ('[000000000000000000000201000000000100]',
  array([[ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0, -1,  0,  1],
         [ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  1,  0,  0]]),
  (0, 0),
  1,
  1,
  -1),
 ('[000000000000000000000102000000000201]',
  array([[ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  1,  0, -1],
  

In [57]:
g.history[:2]
def reward_func(history, decay_factor = .9):
    winner = history[-1]
    nn_inputs = []
    rewards = []
    for boardposition in history[-2::-1]:
        nn_inputs.append(boardposition[1])
        rewards.append(winner)
        winner *= decay_factor
    return nn_inputs, rewards
a = reward_func(g.history)
print(type(a[0]), type(a[1]), len(a[0]), len(a[1]))
x = np.array([boardstate_to_cnn_input(bs) for bs in a[0]])
Y = np.array(a[1])
print(x.shape, Y.shape)

nn_1.update_model(x,Y)

<class 'list'> <class 'list'> 32 32
(32, 6, 6, 2) (32,)
Epoch 1/1
32/32 [==============================] - 1s 41ms/step - loss: 0.1705


In [73]:
def little_sim(agents):
    agent1, agent2 = agents
    g = pentago()
    while g.gameover == False:
        agent1.make_move(g)
        if g.gameover ==True: break
        agent2.make_move(g)
    #print('gameover.')
    return g
#little_sim((agent1,agent2))
#if __name__ == '__main__':
#    with Pool(1) as p:
#        game_returns = p.map(little_sim, [(agent1,agent2)]*12)

In [48]:
def test(x):
    return x, str(time.time())[-1], x*time.time()

if __name__ == '__main__':
    with Pool(6) as p:
        returns = p.map(test, [x for x in range(10)])
print(returns)

[(0, '9', 0.0), (1, '6', 1607202463.258475), (2, '8', 3214404926.517396), (3, '2', 4821607389.776707), (4, '6', 6428809853.036744), (5, '9', 8036012316.29691), (6, '6', 9643214779.55541), (7, '3', 11250417242.814806), (8, '4', 12857619706.074352), (9, '2', 14464822169.333754)]


In [153]:
game_returns = [[1,2,3],[23,24,52],[66]]
games_list = []
for gl in game_returns:
    games_list += gl
games_list

[1, 2, 3, 23, 24, 52, 66]

In [162]:
def big_sim_parallel_nn(n_steps = 1, games_per_step = 32, nn_to_update = [], parallel_threads = 6):
    simulation_times = []
    nn_update_times = []
    winner_list = []
    
    agent1_epsilon = .5
    agent1_epsilon_decay = .99 
    agent1_epsilon_min = .1
    agent1_nn = nn_to_update[0]
    agent1_nn.model.save_weights('nn1_weights.h5')
    agent2_epsilon = .5
    agent2_epsilon_decay = .995 
    agent2_epsilon_min = .2
    agent2_nn = nn_to_update[1]
    agent2_nn.model.save_weights('nn2_weights.h5')

    
    for n in range(n_steps):
        
        #define parallelized process, 
        def parallel_func(reps):
            nn1 = cnn_model(weights = 'nn1_weights.h5')
            nn2 = cnn_model(weights = 'nn2_weights.h5')
            agent1 = nn_agent(player = 1, nn = nn1, epsilon = agent1_epsilon)
            agent2 = nn_agent(player = -1, nn = nn2, epsilon = agent2_epsilon)

            game_list = []
            
            for x in reps:
                g = pentago()
                while g.gameover == False:
                    agent1.make_move(g)
                    if g.gameover ==True: break
                    agent2.make_move(g)
                game_list.append(g)
            
            return games_list
        
        
        print('game_step', n, end = ' ')
        game_start = time.time()
        
        if __name__ == '__main__':
            with Pool(parallel_threads) as p:
                game_returns = p.map(parallel_func, [games_per_step//parallel_threads]*games_per_step)
        #game_returns = [little_sim((agent1,agent2)) for x in range(games_per_step)] #comment our parallelilization if needed.
            
        game_times.append(time.time()-game_start)
        
        # Create single list for iterating
        games_list = []
        for gl in game_returns:
            games_list += gl

        player1_winner = 0
        player2_winner = 0
        # check for winner and create update batch
        nn_input_batch = []
        rewards_for_batch = []
        for game in game_returns:
            if game.winner:
                if game.winner == 1: player1_winner += 1
                else: player2_winner += 1
                
                #cumulate rewards
                boardstates, rewards = reward_func(game.history)
                nn_input_batch += [boardstate_to_cnn_input(bs) for bs in boardstates] # add nn_inputs to training list (x)
                rewards_for_batch += rewards # add rewards to training list (Y)
                
        # train the neural network
        t0 = time.time()
        print(f"updating with {len(nn_input_batch)}, {len(rewards_for_batch)} training batch.")

        agent1_nn.update_model(np.array(nn_input_batch), np.array(rewards_for_batch))
        agent1_nn.model.save_weights(nn1_weights)
        agent2_nn.update_model(np.array(nn_input_batch), np.array(rewards_for_batch))
        agent2_nn.model.save_weights(nn2_weights)
        nn_update_times.append(time.time() - t0)
        
        if agent1.epsilon > agent1.epsilon_min: agent1.epsilon *= agent1.epsilon_decay
        if agent2.epsilon > agent2.epsilon_min: agent2.epsilon *= agent2.epsilon_decay
        print("player 1 wins:", player1_winner)
        print("player 2 wins:", player2_winner)
        print("parallelized batch took", game_times[-1], "seconds.")
        print("neural network update:", nn_update_times[-1], "seconds.")
        
    # end of simulation runs, save q_table(s) to disk
    nn_num = 1
    time_str = str(datetime.now())[:19].replace(':','_')
    for nn in nn_to_update:
        with open(f'pCNN_{nn_num}_'+time_str+'.pickle', 'wb') as file:
            pickle.dump(nn, file, protocol = pickle.HIGHEST_PROTOCOL)
        nn_num += 1
    
    return game_times
            

In [163]:
# Note you will overwrite this q_table and agents if you run this cell again.    Verify you won't lose your data!
#with open('decay_q_table1_2020-11-29 12_09_00.pickle', 'rb') as file:
#    qtable1 =  pickle.load(file)
nn_1 = cnn_model()
nn_2 = cnn_model()
agent1 = nn_agent(player = 1,  nn = nn_1, epsilon_min = .02, epsilon = 1)
agent2 = nn_agent(player = -1,  nn = nn_1, epsilon_min = .15, epsilon = 1)

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 2, 2, 64)          1216      
_________________________________________________________________
flatten_22 (Flatten)         (None, 256)               0         
_________________________________________________________________
dense_64 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_65 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_66 (Dense)             (None, 1)                 129       
Total params: 100,033
Trainable params: 100,033
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_25"
_________________________________________________________________
Layer (type)       

In [164]:
big_sim_parallel_nn(n_steps=1, games_per_step=32, nn_to_update=[nn_1, nn_2], parallel_threads=4)

game_step 0 

AttributeError: Can't pickle local object 'big_sim_parallel_nn.<locals>.parallel_func'

In [79]:
agent2.epsilon

0.9

In [77]:
##################################################
## Change number of games to simulate here
#_games = 10000
##################################################

time0 = time.time()
for x in range(20):
    big_sim_parallel_nn(agent1, agent2, n_steps=10, games_per_step=32, nn_to_update=[nn_1], parallel_threads=6)
print(time.time()-time0, 'seconds.')

game_step 0 updating with {len(nn_input_batch)}, {len(rewards_for_batch)} training batch.
Epoch 1/1
707/707 [==============================] - 0s 177us/step - loss: 0.2039
player 1 wins: 19
player 2 wins: 10
parallelized batch took 56.845152139663696 seconds.
neural network update: 0.12768101692199707 seconds.
game_step 1 updating with {len(nn_input_batch)}, {len(rewards_for_batch)} training batch.
Epoch 1/1
850/850 [==============================] - 0s 169us/step - loss: 0.1874
player 1 wins: 19
player 2 wins: 12
parallelized batch took 60.10708212852478 seconds.
neural network update: 0.145676851272583 seconds.
game_step 2 updating with {len(nn_input_batch)}, {len(rewards_for_batch)} training batch.
Epoch 1/1
775/775 [==============================] - 0s 181us/step - loss: 0.1691
player 1 wins: 25
player 2 wins: 6
parallelized batch took 57.84349822998047 seconds.
neural network update: 0.14233803749084473 seconds.
game_step 3 updating with {len(nn_input_batch)}, {len(rewards_for_bat

KeyboardInterrupt: 

In [23]:
59738/60/60


16.593888888888888

In [14]:
len(qtable1.q_dict)
#agent1.epsilon

5444204

x = [2,2,2,2,2]

with open('test.pickle', 'wb') as file:
    pickle.dump(x, file, protocol = pickle.HIGHEST_PROTOCOL)
    
with open('test.pickle', 'rb') as file:
    y = pickle.load(file)

print(y)

In [15]:
ns = []
for k,v in qtable1.q_dict.items():
    ns.append(v[1])


In [16]:
np.histogram(np.array(ns))

(array([5444199,       3,       1,       0,       0,       0,       0,
              0,       0,       1]),
 array([1.000000e+00, 1.797250e+04, 3.594400e+04, 5.391550e+04,
        7.188700e+04, 8.985850e+04, 1.078300e+05, 1.258015e+05,
        1.437730e+05, 1.617445e+05, 1.797160e+05]))

In [17]:
print(len([x for x in ns if x != 1]))
print(len([x for x in ns if x > 2]))
print(len([x for x in ns if x > 3]))
print(len([x for x in ns if x > 4]))
print(len([x for x in ns if x > 10]))

153873
67959
46602
33784
12230


In [18]:
i = 0
for k,v in qtable1.q_dict.items():
    print(k,v)
    i += 1
    if i > 5: break

[120020100111022211102201112221201221] (0.5, 1)
[020011100211022012102201112221201221] (0.5, 1)
[002011202211010012102201112201201221] (0.5, 1)
[002011202211010012210201010201122221] (0.5, 1)
[020011100211022012210001010201122221] (0.5, 1)
[110020112001200220022012002010111221] (0.5, 1)


In [19]:
np.histogram([x for x in ns if x != 1])

(array([94371,     6,     0,     0,     0,     0,     0,     0,     0,
            1]),
 array([2.00000e+00, 1.15380e+04, 2.30740e+04, 3.46100e+04, 4.61460e+04,
        5.76820e+04, 6.92180e+04, 8.07540e+04, 9.22900e+04, 1.03826e+05,
        1.15362e+05]))